In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM

In [ ]:
from google.colab import files

# Upload CSV files
uploaded = files.upload()

Saving sample_submission.csv to sample_submission.csv
Saving supplemental_clinical_data.csv to supplemental_clinical_data.csv
Saving test.csv to test.csv
Saving test_peptides.csv to test_peptides.csv
Saving test_proteins.csv to test_proteins.csv
Saving train_clinical_data.csv to train_clinical_data.csv
Saving train_peptides.csv to train_peptides.csv
Saving train_proteins.csv to train_proteins.csv


In [ ]:


# Load the data
peptides_df = pd.read_csv('train_peptides.csv')
proteins_df = pd.read_csv('train_proteins.csv')
clinical_df = pd.read_csv('train_clinical_data.csv')

# Merge the peptides and proteins data
merged_df = pd.merge(peptides_df, proteins_df, on=['visit_id', 'visit_month', 'patient_id', 'UniProt'])

# Aggregate the protein abundance data by patient and visit
grouped_df = merged_df.groupby(['patient_id', 'visit_month']).agg({'NPX': 'mean'}).reset_index()

# Merge the clinical data with the protein abundance data
merged_clinical_df = pd.merge(clinical_df, grouped_df, on=['patient_id', 'visit_month'])

# Create a new dataframe with the selected features and target
selected_features = ['NPX']
target = ['updrs_2']

df = merged_clinical_df[['patient_id', 'visit_month'] + selected_features + target].dropna()

# Normalize the features
scaler = StandardScaler()
df[selected_features] = scaler.fit_transform(df[selected_features])

# Split the data into training and holdout sets
train_size = int(0.8 * len(df))
train_df, test_df = df[:train_size], df[train_size:]

# Convert the data into sequences of length 3 for training an LSTM model
def to_sequences(data, sequence_length):
    X = []
    y = []
    
    for i in range(len(data) - sequence_length):
        X.append(data.iloc[i:(i + sequence_length)][selected_features].values)
        y.append(data.iloc[i + sequence_length][target].values)
    
    return np.array(X), np.array(y)

sequence_length = 3

X_train, y_train = to_sequences(train_df, sequence_length)
X_test, y_test = to_sequences(test_df, sequence_length)

# Train the LSTM model
model = Sequential()
model.add(LSTM(128, input_shape=(sequence_length, len(selected_features))))
model.add(Dense(1))

model.compile(loss='mse', optimizer='adam')

model.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test))

# Evaluate the accuracy of the model on the holdout set
y_pred = model.predict(X_test)
y_pred = np.round(y_pred)
accuracy = np.mean(np.abs(y_pred - y_test) <= 10)

print('Accuracy:', accuracy)


Epoch 1/50
27/27 [==============================] - 5s 38ms/step - loss: 68.7763 - val_loss: 56.5268
Epoch 2/50
27/27 [==============================] - 0s 12ms/step - loss: 54.1969 - val_loss: 33.3709
Epoch 3/50
27/27 [==============================] - 0s 8ms/step - loss: 37.2596 - val_loss: 29.7688
Epoch 4/50
27/27 [==============================] - 0s 8ms/step - loss: 36.4561 - val_loss: 29.9424
Epoch 5/50
27/27 [==============================] - 0s 10ms/step - loss: 36.4990 - val_loss: 29.9976
Epoch 6/50
27/27 [==============================] - 0s 8ms/step - loss: 36.5741 - val_loss: 29.8967
Epoch 7/50
27/27 [==============================] - 0s 8ms/step - loss: 36.4497 - val_loss: 29.7649
Epoch 8/50
27/27 [==============================] - 0s 9ms/step - loss: 36.6107 - val_loss: 30.0174
Epoch 9/50
27/27 [==============================] - 0s 8ms/step - loss: 36.6928 - val_loss: 29.6450
Epoch 10/50
27/27 [==============================] - 0s 8ms/step - loss: 36.4127 - val_loss: 29.9